In [2]:
import numpy as np 
import SC_search
import matplotlib.pyplot as plt
import scienceplots 
import matplotlib
matplotlib.rcParams.update({'font.size': 20})
plt.style.use('science')

In [3]:
# Frequency domain setup
year_in_seconds = 365.25*24*60*60

# Frequency bounds
f_low = 0.018
f_high = 0.03



T_obs = (3*year_in_seconds)
d = 1/(T_obs)
safety = 10
cadence = 2/(f_high*safety)

freqs = np.arange(0.015,f_high+d,d)


In [4]:
source_parameters_1 = [28.09555579546043,#mc [sm]
                    0.24710059171597634,#eta
                    50.1*(1.e+6),#D
                    np.pi/4,#beta [rads]
                    2.01,#lambda [rads]
                    2.498091544796509,#inc [rads]
                    -1.851592653589793,#polarization [rads]
                    0,#phi0 [rads]
                    0.018/2,
                    0.01]#e0

source_parameters_2 = [28.09555579546043,#mc [sm]
                    0.24710059171597634,#eta
                    1000.1*(1.e+6),#D
                    np.pi/4,#beta [rads]
                    2.01,#lambda [rads]
                    2.498091544796509,#inc [rads]
                    -1.851592653589793,#polarization [rads]
                    0,#phi0 [rads]
                    0.018/2,
                    0.01]#e0 



In [5]:
source_params_transformed_1 = SC_search.Utility.TaylorF2Ecc_mc_eta_to_m1m2(source_parameters_1.copy())
source_params_transformed_2 = SC_search.Utility.TaylorF2Ecc_mc_eta_to_m1m2(source_parameters_2.copy())


In [6]:
# Use interpolation functionality from BBHx
freqs_sparse = freqs[::1000]
signal_1= SC_search.Waveforms.TaylorF2Ecc.BBHx_response_interpolate_CPU(source_params_transformed_1,
                                                                                  freqs_sparse,
                                                                                  freqs,
                                                                                  f_high,
                                                                                  T_obs,
                                                                                  'AET',
                                                                                  logging=True)
signal_2= SC_search.Waveforms.TaylorF2Ecc.BBHx_response_interpolate_CPU(source_params_transformed_2,
                                                                                  freqs_sparse,
                                                                                  freqs,
                                                                                  f_high,
                                                                                  T_obs,
                                                                                  'AET',
                                                                                  logging=True)




Time to merger is:  3.5550937742535975  years
Upper bound on time in band:  2.6438861012726447  years (no eccentricity evolution assumed)
Time to merger is:  3.5550937742535975  years
Upper bound on time in band:  2.6438861012726447  years (no eccentricity evolution assumed)


In [7]:
Sdisp = SC_search.Noise.Sdisp_SciRD(freqs)
Sopt = SC_search.Noise.Sopt_SciRD(freqs)
psd_A = SC_search.Noise.psd_AEX(freqs,Sdisp,Sopt)
psd_E = SC_search.Noise.psd_AEX(freqs,Sdisp,Sopt)
psd_T = SC_search.Noise.psd_TX(freqs,Sdisp,Sopt)

psd_array = np.array([psd_A,psd_E,psd_T])

noise_A = SC_search.Noise.noise_realization(psd_A,T_obs)
noise_E = SC_search.Noise.noise_realization(psd_E,T_obs)
noise_T = SC_search.Noise.noise_realization(psd_T,T_obs)


data_1 = signal_1 + np.array([noise_A,noise_E,noise_T])
data_2 = signal_2 + np.array([noise_A,noise_E,noise_T])

We have two sources with almost identical source properties except one has a much higher distance than the other. The search statistic is defined as the following: 

$$ \Upsilon = \sum_{j=1}^N \rho_j^2 $$

Where $\rho_j$ is the *phase maximised SNR* in the jth segment. Defined as: 

$$\rho_j = \frac{<s|h_i>}{\sqrt{<h_i|h_i>}} $$

Because $h \sim D^{-1}$, the dependence of $\Upsilon$ on the model distance is divided out! So you dont need to search over distance. But fix it to some reasonable value to avoid numerical instabilities.

In [8]:
SC_search.Semi_Coherent_Functions.upsilon_func(signal_1,data_1,psd_array,d,num_segments=100)

1490.0585909018432

In [9]:
SC_search.Semi_Coherent_Functions.upsilon_func(signal_2,data_1,psd_array,d,num_segments=100)

1490.0585909018432

In [10]:
SC_search.Semi_Coherent_Functions.upsilon_func_masking(signal_2,data_1,psd_array,d,num_segments=100)

1490.0585909018446